In [ ]:
# [ref]
# https://radimrehurek.com/gensim/models/doc2vec.html
# https://github.com/jeradf/bookspace/blob/master/example/doc2vec_moviespace.ipynb

In [27]:
from __future__ import division
from pprint import pprint
import _pickle as cPickle

from gensim.models import Doc2Vec, Phrases
from gensim.models.doc2vec import LabeledSentence
from gensim.matutils import unitvec
import numpy as np
from numpy import dot
import pandas as pd
from random import shuffle
from collections import Counter

from gensim.parsing.preprocessing import STOPWORDS as stop_words
letters = list('abcdefghijklmnopqrstuvwxyz')
numbers = list('123456789')
stop_words = stop_words.union(set(letters)).union(set(numbers))

import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [22]:
df = pd.read_pickle('top_5500_movies_and_shows.df')
df.head()

,title,actors,awards,country,director,genres,first_year,last_year,imdb_id,language,metascore,poster_url,rating,rev_text,type,votes,writer,writers
0,The Shawshank Redemption (1994),"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",NaN,NaN,"[Stephen King, Frank Darabont]","[Crime, Drama]",1994,1994,tt0111161,NaN,NaN,http://ia.media-imdb.com/images/M/MV5BODU4MjU4...,9.3,Why do I want to write the 234th comment on Th...,movie,1684836.0,NaN,"[Stephen King, Frank Darabont]"
1,The Dark Knight (2008),"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",NaN,NaN,"[Jonathan Nolan, Christopher Nolan]","[Action, Crime, Drama, Thriller]",2008,2008,tt0468569,NaN,NaN,http://ia.media-imdb.com/images/M/MV5BMTMxNTMw...,9.0,We've been subjected to enormous amounts of hy...,movie,1670736.0,NaN,"[Jonathan Nolan, Christopher Nolan]"
2,Inception (2010),"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle...",NaN,NaN,[Christopher Nolan],"[Action, Adventure, Sci-Fi, Thriller]",2010,2010,tt1375666,NaN,NaN,http://ia.media-imdb.com/images/M/MV5BMjAxMzY3...,8.8,I'd like to keep my review rather to the point...,movie,1462859.0,NaN,[Christopher Nolan]
3,Fight Club (1999),"[Brad Pitt, Edward Norton, Meat Loaf, Zach Gre...",NaN,NaN,"[Chuck Palahniuk, Jim Uhls]",[Drama],1999,1999,tt0137523,NaN,NaN,http://ia.media-imdb.com/images/M/MV5BNGM2NjQx...,8.8,\nWhen I first saw the previews for this movie...,movie,1347001.0,NaN,"[Chuck Palahniuk, Jim Uhls]"
4,Pulp Fiction (1994),"[John Travolta, Uma Thurman, Samuel L. Jackson...",NaN,NaN,"[Quentin Tarantino, Roger Avary]","[Crime, Drama]",1994,1994,tt0110912,NaN,NaN,http://ia.media-imdb.com/images/M/MV5BMTkxMTA5...,8.9,"One of the early scenes in ""Pulp Fiction"" feat...",movie,1324182.0,NaN,"[Quentin Tarantino, Roger Avary]"


In [28]:
from collections import defaultdict

f_name_plot = open('name_plot.txt', 'r')
#name_plot_dic = defaultdict(list)

name_list = []
plot_list = []

name_plots = f_name_plot.readlines()
for row in name_plots:
    name_plot = row.split('\t')
    #name_plot_dic[name_plot[0]] = name_plot[1]
    if(len(name_plot) == 2):
        name_list.append(name_plot[0])
        plot_list.append(name_plot[1])
    
f_name_plot.close()

In [29]:
import string
import re
RE_PUNCT = re.compile('([%s])+' % re.escape(string.punctuation), re.UNICODE)

def preprocess(text):
    # Remove all punctuation and make all lowercase 
    return RE_PUNCT.sub(" ", text).lower().split()

In [31]:
def make_movie_doc(text, title, drop_stopwords=True):
    """Make documents into LabeledSentence objects for doc2vec training"""
    doctag = '_'.join(preprocess(title))    
    docwords = list(filter(lambda word: word not in stop_words,
                      bigram[preprocess(text)]))
    return LabeledSentence(docwords, [doctag])

# Train bigrammer to detect two-word phrases, e.g., breaking_bad
#bigram = Phrases(map(preprocess, df.rev_text.tolist())) 
bigram = Phrases(map(preprocess, plot_list)) 

#DOCS = [make_movie_doc(text, title) for text, title in
#        zip(df.rev_text.tolist(), df.title.tolist())]
DOCS = [make_movie_doc(text, title) for text, title in
        zip(plot_list, name_list)]



shuffle(DOCS)

2017-06-15 12:58:00,277 : INFO : collecting all words and their counts
2017-06-15 12:58:00,281 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2017-06-15 12:58:06,966 : INFO : collected 1529038 word types from a corpus of 4813925 words (unigram + bigrams) and 8472 sentences
2017-06-15 12:58:06,966 : INFO : using 1529038 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
/home/hong/anaconda3/lib/python3.5/site-packages/gensim/models/phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [32]:
model = Doc2Vec(dm=0, dbow_words=1, min_count=4, negative=3,
                hs=0, sample=1e-4, window=10, size=100, workers=8)

model.build_vocab(DOCS)
model.train(DOCS, total_examples=model.corpus_count, epochs=model.iter)

2017-06-15 12:58:27,206 : INFO : collecting all words and their counts
2017-06-15 12:58:27,207 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-15 12:58:27,696 : INFO : collected 92200 word types and 8470 unique tags from a corpus of 8472 examples and 2394249 words
2017-06-15 12:58:27,697 : INFO : Loading a fresh vocabulary
2017-06-15 12:58:28,295 : INFO : min_count=4 retains 48512 unique words (52% of original 92200, drops 43688)
2017-06-15 12:58:28,295 : INFO : min_count=4 leaves 2327785 word corpus (97% of original 2394249, drops 66464)
2017-06-15 12:58:28,397 : INFO : deleting the raw counts dictionary of 92200 items
2017-06-15 12:58:28,400 : INFO : sample=0.0001 downsamples 513 most-common words
2017-06-15 12:58:28,400 : INFO : downsampling leaves estimated 2044034 word corpus (87.8% of prior 2327785)
2017-06-15 12:58:28,400 : INFO : estimated required memory for 48512 words and 100 dimensions: 68147600 bytes
2017-06-15 12:58:28,539 : INFO : 

10261902

In [33]:
# Find words similar to query word
pprint(model.most_similar('hero'))

2017-06-15 12:58:59,409 : INFO : precomputing L2-norms of word weight vectors


[('hailed_as', 0.7197465896606445),
 ('heroic', 0.6965265870094299),
 ('heroism', 0.6770747303962708),
 ('weary', 0.6507082581520081),
 ('honored', 0.6473628282546997),
 ('heroes', 0.6452208161354065),
 ('accomplished', 0.638401985168457),
 ('pyrrhic', 0.6361966133117676),
 ('kebbell', 0.6338528990745544),
 ('inspired_by', 0.632212221622467)]


In [34]:
# Find movies similar to query word
vec = model['hero']
pprint(model.docvecs.most_similar([vec]))

2017-06-15 12:59:06,873 : INFO : precomputing L2-norms of doc weight vectors


[('simon_birch_1998', 0.6298867464065552),
 ('blues_brothers_2000_1998', 0.6213065385818481),
 ('great_dictator_the_1940', 0.6204599738121033),
 ('moon_over_parador_1988', 0.6066589951515198),
 ('six_string_samurai_1998', 0.6056036949157715),
 ('dragon_the_bruce_lee_story_1993', 0.6035102009773254),
 ('lenny_1974', 0.6014411449432373),
 ('condemned_the_2007', 0.5961964130401611),
 ('fallen_idol_the_1948', 0.5941512584686279),
 ('honeysuckle_rose_a_k_a_on_the_road_again_1980', 0.5934443473815918)]


In [9]:
pprint(model.docvecs[1])

array([ -2.01952770e-01,  -4.74567771e-01,   4.21493530e-01,
         3.94441038e-01,   2.12652594e-01,   2.74790764e-01,
         4.28851217e-01,  -3.43415499e-01,  -3.13537687e-01,
        -5.66040091e-02,  -2.06435144e-01,   6.66458681e-02,
        -2.82495886e-01,  -2.58645266e-01,   2.34836519e-01,
         4.22616273e-01,  -5.69512069e-01,   4.48908433e-02,
        -1.36964023e-01,  -5.07732511e-01,  -2.62371629e-01,
         1.52882054e-01,   3.52895856e-01,   6.34373307e-01,
        -3.48489642e-01,  -1.74726322e-01,  -8.72953415e-01,
        -5.49116075e-01,  -4.06443834e-01,   4.58787344e-02,
        -3.79901975e-01,   7.24184588e-02,   1.34766370e-01,
        -1.24760069e-01,  -4.88436133e-01,  -3.21606070e-01,
         3.28891248e-01,   1.24825358e+00,   8.26135933e-01,
        -3.37436527e-01,   1.12053268e-01,   1.40768752e-01,
        -5.69933832e-01,   2.52969474e-01,  -5.93420267e-02,
        -4.86876726e-01,  -4.36180830e-02,   3.61817747e-01,
         2.46026829e-01,

In [66]:
# Find movies similar to a query movie
#for title in df.title.tolist()[:1]:
targetMovie = ['Glory (1989)', 'Boogie Nights (1997)', 'You\'ve Got Mail (1998)', \
               'I, Robot (2004)', 'Walk the Line (2005)', 'Black Hawk Down (2001)', \
               '300 (2007)', 'Superbad (2007)', 'Cars 3 (2017)']
#for title in name_list[790:791]:
for title in targetMovie:
    print("\nQuery: %s"%title)
    doctag = '_'.join(preprocess(title))
    pprint(model.docvecs.most_similar(doctag))



Query: Glory (1989)
[('cross_of_iron_1977', 0.8848959803581238),
 ('last_samurai_the_2003', 0.8841438293457031),
 ('lawrence_of_arabia_1962', 0.8840969204902649),
 ('sergeant_york_1941', 0.8820092678070068),
 ('rio_grande_1950', 0.8784996271133423),
 ('we_were_soldiers_2002', 0.876758337020874),
 ('patton_1970', 0.8763837814331055),
 ('they_were_expendable_1945', 0.8700815439224243),
 ('enemy_at_the_gates_2001', 0.8669483661651611),
 ('patriot_the_2000', 0.866405725479126)]

Query: Boogie Nights (1997)
[('frogs_for_snakes_1998', 0.8685746192932129),
 ('car_wash_1976', 0.8356642723083496),
 ('empire_2002', 0.8300975561141968),
 ('broken_hearts_club_the_2000', 0.8214051723480225),
 ('keys_to_tulsa_1997', 0.8202093243598938),
 ('bobby_2006', 0.8180745244026184),
 ('owning_mahowny_2003', 0.8179845809936523),
 ('smile_1975', 0.8169777393341064),
 ('middle_men_2009', 0.8127021789550781),
 ('hearts_of_the_west_1975', 0.8116779327392578)]

Query: You've Got Mail (1998)
[('this_is_my_life_1992

In [67]:
# Find words similar to a query movie
targetMovie = ['Glory (1989)', 'Boogie Nights (1997)', 'You\'ve Got Mail (1998)', \
               'I, Robot (2004)', 'Walk the Line (2005)', 'Black Hawk Down (2001)', \
               '300 (2007)', 'Superbad (2007)', 'Cars 3 (2017)']
#for title in name_list[790:791]:
for title in targetMovie:
    print("\nQuery: %s"%title)
    doctag = '_'.join(preprocess(title))
    vec = model.docvecs[doctag]
    pprint(model.most_similar([vec]))


Query: Glory (1989)
[('54th', 0.9147264361381531),
 ('regimental', 0.9103386998176575),
 ('eastern_front', 0.9068944454193115),
 ('stransky', 0.9046857357025146),
 ('battalion', 0.9004204273223877),
 ('zulu', 0.8941487669944763),
 ('regiment', 0.8896756768226624),
 ('wehrmacht', 0.8896723389625549),
 ('north_africa', 0.8894959092140198),
 ('jin_tae', 0.8863554000854492)]

Query: Boogie Nights (1997)
[('matt_dillon', 0.8894107937812805),
 ('robb', 0.8807612657546997),
 ('hustler', 0.877149760723114),
 ('shea', 0.874844491481781),
 ('snorting', 0.8693946003913879),
 ('southern_california', 0.8691381812095642),
 ('showgirl', 0.8676514625549316),
 ('anthony_lapaglia', 0.8666927814483643),
 ('clifton', 0.8661743402481079),
 ('wannabe', 0.8611297607421875)]

Query: You've Got Mail (1998)
[('malin', 0.9109910726547241),
 ('greeting_card', 0.893404483795166),
 ('stylish', 0.8887248635292053),
 ('fancies', 0.8846456408500671),
 ('record_store', 0.8842371106147766),
 ('melanie_griffith', 0.8837

In [58]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from tsne import bh_sne

def pca_transform_vecs(vecs, n=50):
    from sklearn.decomposition import PCA
    pca = PCA(n_components=n)
    pca.fit(vecs)
    return pca.transform(vecs)

def tsne_plot(titles, dims=40, perplexity=6, save_as=''):
    doctags = ['_'.join(preprocess(title)) for title in titles]
    vecs = np.array([unitvec(model.docvecs[doctag]) for doctag in doctags])    
    # First doing pca on the vectors can reduce the noise and yield a better
    # 2d projection
    small_vecs = pca_transform_vecs(vecs, dims)
    tsne_vecs = bh_sne(small_vecs, perplexity=perplexity)

    fig, ax = plt.subplots(figsize=(18, 18))
    ax.scatter(tsne_vecs[:,0], tsne_vecs[:,1])

    # Annotate points with the movie title
    for i, title in enumerate(titles):
        ax.annotate(title, 
                    xy=(tsne_vecs[i,0],tsne_vecs[i,1]), 
                    fontsize=12, alpha=.9)

    plt.xlim(min(tsne_vecs[:,0])-0.3, max(tsne_vecs[:,0])+0.3) 
    plt.ylim(min(tsne_vecs[:,1])-0.3, max(tsne_vecs[:,1])+0.3)
    if save_as:
        fig.savefig(save_as, dpi=fig.dpi)
    return fig

ImportError: No module named 'tsne'

In [ ]:
TOP_N = 90
query = 'type=="series" and rating>7.0'
titles = df.query(query).title.tolist()[:TOP_N]
fig = tsne_plot(titles, save_as='tv_shows.png')

In [ ]:
query = 'type=="movie" and rating>7.0'
titles = df.query(query).title.tolist()[:TOP_N]
fig = tsne_plot(titles, save_as='movies.png')